<a href="https://colab.research.google.com/github/markpairdha/Face-mask-detection/blob/main/face_mask_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt
from imutils import paths
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [3]:
DIRECTORY = r"/content/drive/MyDrive/Colab Notebooks/Face-Mask-Detection"
CATEGORIES = ["with_mask", "without_mask"]  # 1915 #1918

In [4]:
data = []
labels = []

for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
    	img_path = os.path.join(path, img)
    	image = load_img(img_path, target_size=(224, 224))
    	image = img_to_array(image)
    	image = preprocess_input(image)

    	data.append(image)
    	labels.append(category)

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [5]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [6]:
data = np.array(data, dtype="float32")
labels = np.array(labels)

In [7]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.20, stratify=labels, random_state=42)

In [8]:
# construct the training image generator for producing many images
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [9]:
# load the MobileNetV2 network
baseModel = MobileNetV2(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

9412608/9406464 [==============================] - 0s 0us/step


In [10]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)

In [11]:
# leaving imagenet weights as it is! 
for layer in baseModel.layers:
	layer.trainable = False

In [12]:
# compiling model
model.compile(loss="binary_crossentropy", optimizer=Adam(lr=1e-4, decay=1e-4/20),metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [14]:
# train the head of the network
H = model.fit(
	aug.flow(trainX, trainY, batch_size=32),
	steps_per_epoch=len(trainX) // 32,
	validation_data=(testX, testY),
	validation_steps=len(testX) // 32,
	epochs=20)

Epoch 1/20
95/95 [==============================] - 131s 1s/step - loss: 0.4490 - accuracy: 0.8355 - val_loss: 0.1809 - val_accuracy: 0.9765
Epoch 2/20
95/95 [==============================] - 127s 1s/step - loss: 0.1583 - accuracy: 0.9628 - val_loss: 0.1045 - val_accuracy: 0.9739
Epoch 3/20
95/95 [==============================] - 128s 1s/step - loss: 0.1066 - accuracy: 0.9730 - val_loss: 0.0674 - val_accuracy: 0.9831
Epoch 4/20
95/95 [==============================] - 127s 1s/step - loss: 0.0796 - accuracy: 0.9786 - val_loss: 0.0556 - val_accuracy: 0.9831
Epoch 5/20
95/95 [==============================] - 127s 1s/step - loss: 0.0648 - accuracy: 0.9832 - val_loss: 0.0476 - val_accuracy: 0.9844
Epoch 6/20
95/95 [==============================] - 127s 1s/step - loss: 0.0580 - accuracy: 0.9809 - val_loss: 0.0427 - val_accuracy: 0.9870
Epoch 7/20
95/95 [==============================] - 127s 1s/step - loss: 0.0479 - accuracy: 0.9875 - val_loss: 0.0389 - val_accuracy: 0.9883
Epoch 8/20
95

In [15]:
# make predictions on the testing set
predict = model.predict(testX, batch_size=32)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predict = np.argmax(predict, axis=1)

print(classification_report(testY.argmax(axis=1), predict,target_names=lb.classes_))
model.save("mask_detector.model", save_format="h5")

              precision    recall  f1-score   support

   with_mask       0.99      0.99      0.99       383
without_mask       0.99      0.99      0.99       384

    accuracy                           0.99       767
   macro avg       0.99      0.99      0.99       767
weighted avg       0.99      0.99      0.99       767



/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
